In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from adjustText import adjust_text
import sklearn.metrics as metrics

# word2vec
import gensim
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# preprocess
import datetime
import statistics 

import json, requests

In [2]:
data = pd.read_csv("/gpfs/data/iime/data/admissions/adm_research_aptitude.csv")
data = data.applymap(str)
# data.head(10)
data.head()
# len(data)

,AMCAS_ID,APP_YEAR,EXPERIENCE_NAME,EXPERIENCE_DESCR,MEANINGFUL_EXPERIENCE_DESCR,ORGANIZATION_NAME,START_DATE,END_DATE,TOTAL_HOURS,FACULTY_ID,APTITUDE_SCORE,APPLICATION_RANK
0,10792888,2014,Independent Study and Research Assistant - Duk...,"(Full-time paid position during the summers, 1...",nan,"Duke Univ Hospital, Div of Infectious Diseases...",1-Jun-98,1-Jul-98,340.0,1839.0,1.0,3.0
1,10792888,2014,Independent Study and Research Assistant - Duk...,"(Full-time paid position during the summers, 1...",nan,"Duke Univ Hospital, Div of Infectious Diseases...",1-Jun-98,1-Jul-98,340.0,1866.0,3.0,3.0
2,10792888,2014,Independent Study and Research Assistant - Duk...,"(Full-time paid position during the summers, 1...",nan,"Duke Univ Hospital, Div of Infectious Diseases...",1-Aug-97,1-May-98,600.0,1839.0,1.0,3.0
3,10792888,2014,Independent Study and Research Assistant - Duk...,"(Full-time paid position during the summers, 1...",nan,"Duke Univ Hospital, Div of Infectious Diseases...",1-Aug-98,1-May-99,600.0,1866.0,3.0,3.0
4,10792888,2014,Independent Study and Research Assistant - Duk...,"(Full-time paid position during the summers, 1...",nan,"Duke Univ Hospital, Div of Infectious Diseases...",1-Jun-99,1-Jul-99,340.0,1839.0,1.0,3.0


In [3]:
recommenders = pd.read_csv("/gpfs/data/iime/erica_chio/thesis/recommenders.csv")
recommenders.head()
recommenders = recommenders.rename(columns={"AAMC_ID": "AMCAS_ID"})
 
# len(recommenders)

In [4]:
from src import preprocess_research as preprocess_research
data = preprocess_research.process_reg(data, 'EXPERIENCE_DESCR')
backup = data

finished processing,  13859  rows
1:0 ratio =  0.23169059816725593 : 0.7683094018327441


In [5]:
data = backup

In [6]:
data['AMCAS_ID'] = data['AMCAS_ID'].astype(int)
df = pd.merge(data, recommenders)

In [7]:
df.head()

,AMCAS_ID,APP_YEAR,ORGANIZATION_NAME,FACULTY_ID,MONTHS,EXPERIENCE_DESCR,TOTAL_HOURS,APTITUDE_SCORE,UNIQUE_EXP,MONTHS3,MONTHS6,MONTHS12,PUBLICATIONS,app_year,application_id,title,address,email
0,10989856,2014,"[hepatobiliary, pancreatic, and transplant sur...","[12091.0, 12095.0]",[2],i began my first summer of research going thro...,1280.0,0,2.0,0,0,0,1.0,2014,11133,Dean of Academic Services,"415 South Street / Usdan 130, MS 001",godsoe@brandeis.edu
1,11008482,2015,[harvard university],"[399.0, 5053.0]",[57],i completed my graduate studies through the ha...,34000.0,1,1.0,0,0,1,2.0,2015,20402,Allston Burr Resident Dean,Dunster House / Office of the Allston Burr Sen...,cdiaz@fas.harvard.edu
2,11378724,2015,"[columbia university, department of microbiolo...","[19371.0, 19367.0]","[0, 17, 3, 20]",as an independent project i helped to charact...,5840.0,0,6.0,1,0,2,0.0,2015,21577,NaN,403 Lewisohn Hall,ss4269@columbia.edu
3,12332712,2017,"[botswana-harvard aids institute partnership, ...","[5050.0, 5190.0]","[10, 11]",i was appointed to a research fellowship in th...,7600.0,1,2.0,0,2,0,2.0,2017,39871,Dean,108 White Gravenor,premed@georgetown.edu
4,12557740,2014,"[bryn mawr college genetics lab, duke universi...","[1845.0, 1039.0]","[2, 4, 8, 10, 11]",i worked for months with dr chapman on a w...,18920.0,0,13.0,1,3,0,1.0,2014,11996,Associate Dean,Health Professions Advising / Canwyll House Ea...,jdomsky@brynmawr.edu


In [8]:
# df['ORGANIZATION_NAME']
# for index, row in df.iterrows():
#     print(len(row.ORGANIZATION_NAME), row.ORGANIZATION_NAME)

In [9]:
url = "https://raw.githubusercontent.com/Hipo/university-domains-list/master/world_universities_and_domains.json"
# response = urllib.urlopen(url)
# schools = json.loads(response.read())
# print (schools)
edu_domains = requests.get(url).json()

In [10]:
def replace_words(a, name):
    result = name
    # if city not in name, drop ? 
    if "college" in name:
        result = name.replace("college", "").strip()
    elif "university" in name:
        result = name.replace("university", "").strip()
    a.append(result)
    
    return list(set(a))


In [11]:
def find_domain(domain):
#     print(i.get('domains')[0].split(".")[0], i.get("name"))
    for i in edu_domains:
        all_domains = i.get('domains')
        for each in all_domains:
            if each.split(".")[0] == domain:
                if "&" in i.get("name").lower(): 
                    return (i.get("name").lower().replace("&", "and"))
                else:
                    return (i.get("name").lower())
    return False


In [12]:
find_domain("jhu")

# find_domain("yale-nus")

'johns hopkins university'

In [13]:
df = df.dropna()
school = []
for index, row in df.iterrows():
    if row.email == 'nilanjana.pal@yale-nus.edu.sg': # this email infinitely loops 
#         print("idk whats happening")
        s.append("yale-nus")
        s.append("national university of singapore")
        school.append(s)
    else:
        loc = row.email.split("@")[-1].split(".")
        s = loc[: len(loc)-1]
        s = [es.lower() for es in s]
#         print(row.email, s)
#         s.append("what")
        for e in s:
            full_name = find_domain(e)
            if full_name != False:
                print(e, full_name)
                s.append(full_name)
#                 s = replace_words(s, full_name)
        school.append(s)
    print(index, s, row.email)
    
df['school'] = school
# df['school'] = df['school'].str.replace('[^a-zA-Z]',' ').str.lower()

brandeis brandeis university
0 ['brandeis', 'brandeis university'] godsoe@brandeis.edu
harvard harvard university
1 ['fas', 'harvard', 'harvard university'] cdiaz@fas.harvard.edu
georgetown georgetown university
3 ['georgetown', 'georgetown university'] premed@georgetown.edu
brynmawr bryn mawr college
4 ['brynmawr', 'bryn mawr college'] jdomsky@brynmawr.edu
owu ohio wesleyan university
5 ['owu', 'ohio wesleyan university'] ajgatz@owu.edu
owu ohio wesleyan university
6 ['owu', 'ohio wesleyan university'] ajgatz@owu.edu
sfsu san francisco state university
10 ['sfsu', 'san francisco state university'] brothman@sfsu.edu
upenn university of pennsylvania
11 ['upenn', 'university of pennsylvania'] chagan@upenn.edu
brynmawr bryn mawr college
14 ['brynmawr', 'bryn mawr college'] jdomsky@brynmawr.edu
brynmawr bryn mawr college
15 ['brynmawr', 'bryn mawr college'] jdomsky@brynmawr.edu
brown brown university
16 ['brown', 'brown university'] George_Vassilev@brown.edu
hunter hunter college
cuny city

harvard harvard university
220 ['fas', 'harvard', 'harvard university'] shamel@fas.harvard.edu
dartmouth dartmouth college
224 ['dartmouth', 'dartmouth college'] sarah.l.berger@dartmouth.edu
dartmouth dartmouth college
225 ['dartmouth', 'dartmouth college'] Sarah.l.berger@dartmouth.edu
nyu new york university
226 ['nyu', 'new york university'] preprof.docs@nyu.edu
wesleyan wesleyan university
228 ['wesleyan', 'wesleyan university'] pcbest@wesleyan.edu
princeton princeton university
229 ['princeton', 'princeton university'] hpa@princeton.edu
ua university of alabama at tuscaloosa
230 ['ua', 'university of alabama at tuscaloosa'] premed@ua.edu
cornell cornell university
231 ['cornell', 'cornell university'] hcec@cornell.edu
nyu new york university
234 ['nyu', 'new york university'] preprof.docs@nyu.edu
brown brown university
237 ['brown', 'brown university'] George_Vassilev@brown.edu
neu northeastern university
238 ['neu', 'northeastern university'] j.woolfson@neu.edu
pitt university of 

452 ['cornell', 'cornell university'] hcec@cornell.edu
cornell cornell university
453 ['cornell', 'cornell university'] hcec@cornell.edu
brown brown university
454 ['brown', 'brown university'] george_vassilev@brown.edu
rice rice university
456 ['rice', 'rice university'] noreen@rice.edu
bowdoin bowdoin college
458 ['bowdoin', 'bowdoin college'] sramus@bowdoin.edu
fordham fordham university
460 ['fordham', 'fordham university'] ewatts1@fordham.edu
williams williams college
461 ['williams', 'williams college'] Jane.D.Cary@williams.edu
brown brown university
462 ['brown', 'brown university'] George_Vassilev@Brown.edu
swarthmore swarthmore college
463 ['swarthmore', 'swarthmore college'] gsimeon1@swarthmore.edu
brown brown university
466 ['brown', 'brown university'] George_Vassilev@Brown.edu
jhu johns hopkins university
469 ['jhu', 'johns hopkins university'] dverrie1@jhu.edu
upenn university of pennsylvania
471 ['upenn', 'university of pennsylvania'] trothman@upenn.edu
wustl washington 

628 ['utdallas', 'university of texas at dallas'] karen.deolivares@utdallas.edu
brown brown university
629 ['brown', 'brown university'] george_vassilev@brown.edu
dartmouth dartmouth college
632 ['dartmouth', 'dartmouth college'] Sarah.L.Berger@dartmouth.edu
harvard harvard university
633 ['fas', 'harvard', 'harvard university'] mallins@fas.harvard.edu 
jhu johns hopkins university
634 ['jhu', 'johns hopkins university'] alextan@jhu.edu
jhu johns hopkins university
635 ['jhu', 'johns hopkins university'] alextan@jhu.edu
636 ['scrippscollege'] dmulay@scrippscollege.edu
columbia columbia university
637 ['columbia', 'columbia university'] mr2168@columbia.edu
harvard harvard university
639 ['fas', 'harvard', 'harvard university'] ma-abrd@fas.harvard.edu
dartmouth dartmouth college
641 ['dartmouth', 'dartmouth college'] Annette.L.Hamilton@dartmouth.edu
harvard harvard university
642 ['fas', 'harvard', 'harvard university'] lo-abrd@fas.harvard.edu
wesleyan wesleyan university
643 ['wesleyan'

harvard harvard university
811 ['fas', 'harvard', 'harvard university'] leafgren@fas.harvard.edu
harvard harvard university
813 ['fas', 'harvard', 'harvard university'] leafgren@fas.harvard.edu
brown brown university
814 ['brown', 'brown university'] george_vassilev@brown.edu
upenn university of pennsylvania
815 ['upenn', 'university of pennsylvania'] trothman@upenn.edu
tufts tufts university
816 ['tufts', 'tufts university'] carol.baffi-dugan@tufts.edu
sas institute of advanced legal studies, university of london
upenn university of pennsylvania
817 ['sas', 'upenn', 'institute of advanced legal studies, university of london', 'university of pennsylvania'] vrdorn@sas.upenn.edu
sas institute of advanced legal studies, university of london
upenn university of pennsylvania
818 ['sas', 'upenn', 'institute of advanced legal studies, university of london', 'university of pennsylvania'] jmclaugh@sas.upenn.edu
uchicago university of chicago
821 ['uchicago', 'university of chicago'] krapec@uchi

...

6555 ['nyu', 'new york university'] preprof.docs@nyu.edu
yale yale university
6556 ['yale', 'yale university'] kristin.mcjunkins@yale.edu
bu boston university
6560 ['bu', 'boston university'] gzamansk@bu.edu
6570 ['gmail'] hunter.phcl@gmail.com
caltech california institute of technology
6574 ['caltech', 'california institute of technology'] jberk@caltech.edu
bu boston university
6575 ['bu', 'boston university'] gzamansk@bu.edu
yu yeshiva university
6576 ['yu', 'yeshiva university'] woodhill@yu.edu
dartmouth dartmouth college
6584 ['dartmouth', 'dartmouth college'] Sarah.L.Berger@Dartmouth.edu
columbia columbia university
6585 ['columbia', 'columbia university'] njc2001@columbia.edu
jhu johns hopkins university
6587 ['jhu', 'johns hopkins university'] dverrie1@jhu.edu
miami university of miami
6589 ['miami', 'university of miami'] prehealth@miami.edu
williams williams college
6592 ['williams', 'williams college'] bf6@williams.edu
njit new jersey institute of technology
6595 ['njit', 'ne

6796 ['bc', 'boston college'] premed@bc.edu
columbia columbia university
6799 ['columbia', 'columbia university'] bma2127@columbia.edu
upenn university of pennsylvania
6800 ['exchange', 'upenn', 'university of pennsylvania'] chagan@exchange.upenn.edu
swarthmore swarthmore college
6801 ['swarthmore', 'swarthmore college'] gsimeon1@swarthmore.edu
jhu johns hopkins university
6802 ['jhu', 'johns hopkins university'] kelli.johnson@jhu.edu 
bc boston college
6806 ['bc', 'boston college'] premed@bc.edu
harvard harvard university
6807 ['fas', 'harvard', 'harvard university'] adamshousedean@fas.harvard.edu
emory emory university
6808 ['emory', 'emory university'] sobrent@emory.edu
case case western reserve university
6812 ['case', 'case western reserve university'] sps17@case.edu
emory emory university
6813 ['emory', 'emory university'] sobrent@emory.edu
lehigh lehigh university
6815 ['lehigh', 'lehigh university'] mer2@lehigh.edu
binghamton state university of new york at binghamton
6821 ['bi

7067 ['columbia', 'columbia university'] njc2001@columbia.edu
harvard harvard university
7069 ['fas', 'harvard', 'harvard university'] lowellhousedean@fas.harvard.edu 
bennington bennington college
7070 ['bennington', 'bennington college'] jfoley@bennington.edu
jhu johns hopkins university
7071 ['jhu', 'johns hopkins university'] kelli.johnson@jhu.edu
rice rice university
7072 ['rice', 'rice university'] hpa@rice.edu
wlu washington and lee university
7073 ['wlu', 'washington and lee university'] altyl@wlu.edu
georgetown georgetown university
7075 ['georgetown', 'georgetown university'] premed@georgetown.edu
princeton princeton university
7076 ['princeton', 'princeton university'] hpa@princeton.edu
upenn university of pennsylvania
7080 ['upenn', 'university of pennsylvania'] trothman@upenn.edu
upenn university of pennsylvania
7081 ['upenn', 'university of pennsylvania'] trothman@upenn.edu 
bc boston college
7084 ['bc', 'boston college'] premed@bc.edu
jhu johns hopkins university
7087 ['

7273 ['brandeis', 'brandeis university'] stutz@brandeis.edu
jhu johns hopkins university
7274 ['jhu', 'johns hopkins university'] kelli.johnson@jhu.edu
brynmawr bryn mawr college
7275 ['brynmawr', 'bryn mawr college'] hpao@brynmawr.edu
brandeis brandeis university
7278 ['brandeis', 'brandeis university'] stutz@brandeis.edu
swarthmore swarthmore college
7279 ['swarthmore', 'swarthmore college'] gsimeon1@swarthmore.edu
7283 ['gmail'] hunter.phcl@gmail.com
emory emory university
7284 ['emory', 'emory university'] sobrent@emory.edu
harvard harvard university
7285 ['fas', 'harvard', 'harvard university'] pforz@fas.harvard.edu
dartmouth dartmouth college
7287 ['dartmouth', 'dartmouth college'] Sarah.L.Berger@dartmouth.edu
nyu new york university
7291 ['nyu', 'new york university'] preprof.doc@nyu.edu
dartmouth dartmouth college
7293 ['dartmouth', 'dartmouth college'] Sarah.L.Berger@Dartmouth.edu
7295 ['kecksci', 'claremont'] jjones@kecksci.claremont.edu
umass university of massachusetts at a

7552 ['georgetown', 'georgetown university'] premed@georgetown.edu
hofstra hofstra university
7556 ['hofstra', 'hofstra university'] Ellen.C.Miller@hofstra.edu
jhu johns hopkins university
7558 ['jhu', 'johns hopkins university'] kelli.johnson@jhu.edu
uchicago university of chicago
7559 ['uchicago', 'university of chicago'] krapec@uchicago.edu
memphis university of memphis
7560 ['memphis', 'university of memphis'] jgclffrd@memphis.edu
jhu johns hopkins university
7561 ['jhu', 'johns hopkins university'] kelli.johnson@jhu.edu
dartmouth dartmouth college
7562 ['dartmouth', 'dartmouth college'] annette.l.hamilton@dartmouth.edu
nyu new york university
7563 ['nyu', 'new york university'] preprof.docs@nyu.edu
tufts tufts university
7564 ['tufts', 'tufts university'] hprc@tufts.edu
dartmouth dartmouth college
7565 ['dartmouth', 'dartmouth college'] Sarah.L.Berger@Dartmouth.edu
buffalo state university of new york at buffalo
7568 ['buffalo', 'state university of new york at buffalo'] etm3@buff

In [14]:
count = 0
total = 0
for index, row in df. iterrows():
#     print("---one entry---")
    total += 1
    counted = False
    for school in row.school:
        for place in row.ORGANIZATION_NAME:
            if school in place and counted == False:
                counted = True
                count += 1
#                 print("counted 1", place, " : " , school)
#                 print(place, school) 
    if counted == False:
        print(row.ORGANIZATION_NAME, " : " , row.school)
        print("-------")
    

print("matched: ", (count/total))

# basic match .5068316576437226 # double counts  - cuny / hunter , rice 
# lower case all 0.5326114978087136
# prevent double count 0.43696829079659705 - cuny / hunter still double count
# one / entry matched:  0.43335911317349834
# names expanded, 0.5522041763341067
# & to and, 0.5545243619489559
# drop / replace uni/ college 0.5939675174013921 - nyu & ny preb matched 
    # 2 matches at least ie: ["new york presbyterian morgan stanley children's hospital"]  :  ['nyu', 'new york university']


['hepatobiliary, pancreatic, and transplant surgery, usc']  :  ['brandeis', 'brandeis university']
-------
['botswana-harvard aids institute partnership', 'harvard school of public health']  :  ['georgetown', 'georgetown university']
-------
['johns hopkins university']  :  ['owu', 'ohio wesleyan university']
-------
['johns hopkins university']  :  ['owu', 'ohio wesleyan university']
-------
['laboratory of neuro imaging at ucla']  :  ['sfsu', 'san francisco state university']
-------
['nan']  :  ['brynmawr', 'bryn mawr college']
-------
['nan']  :  ['brynmawr', 'bryn mawr college']
-------
['rhode island hospital department of surgical research']  :  ['brown', 'brown university']
-------
['johns hopkins igem', 'johns hopkins', 'tsinghua university']  :  ['jhu', 'johns hopkins university']
-------
['johns hopkins igem', 'johns hopkins', 'tsinghua university']  :  ['jhu', 'johns hopkins university']
-------
['johns hopkins lipid research and atherosclerosis division', 'bloomberg school

-------
['african services committee', 'new york methodist hospital']  :  ['nyu', 'new york university']
-------
['wyss institute for biologically inspired engineering', 'department of chemistry, university of arizona', 'mooney laboratory for cellular and tissue engineering']  :  ['fas', 'harvard', 'harvard university']
-------
['dana farber cancer institute', "brigham and women's hospital", 'national istitutes of health']  :  ['bard', 'bard college']
-------
['university of colorado boulder', 'albert einstein college of medicine']  :  ['caltech', 'california institute of technology']
-------
['university of southern california', 'transplantation biology research center']  :  ['fas', 'harvard', 'harvard university']
-------
['department of applied physics', 'the raymond and beverly sackler institute for biological,']  :  ['yale', 'yale university']
-------
['johns hopkins hospital, wilmer eye institute']  :  ['jhu', 'johns hopkins university']
-------
['the pallas lab']  :  ['emory', '

['north york general hospital, department of pediatrics', "children's hospital at montefiore", 'neurosurgery lab, montefiore medical center']  :  ['yu', 'yeshiva university']
-------
['drazin research group']  :  ['rice', 'rice university']
-------
['capobianco lab - molecular oncology program']  :  ['miami', 'university of miami']
-------
['capobianco lab - molecular oncology program']  :  ['miami', 'university of miami']
-------
['advanced polymer research lab']  :  ['utdallas', 'university of texas at dallas']
-------
['uc davis medical center', 'weill cornell medical college', 'butler hospital']  :  ['brown', 'brown university']
-------
['koel lab']  :  ['princeton', 'princeton university']
-------
["children's healthcare of atlanta"]  :  ['emory', 'emory university']
-------
['medical university of south carolina']  :  ['nyu', 'new york university']
-------
['university of medicine and dentistry of nj (radiology dept)']  :  ['villanova', 'villanova university']
-------
["dr. ranal

['johns hopkins hospital', 'new york presbyterian']  :  ['jhu', 'johns hopkins university']
-------
['punt-emerson lab, herbert irving comprehensive cancer center', 'regeneron pharmaceuticals', 'new york presbyterian hospital']  :  ['columbia', 'columbia university']
-------
['he lab, university of california, berkeley', 'us social security administration, odar/oao']  :  ['sas', 'upenn', 'institute of advanced legal studies, university of london', 'university of pennsylvania']
-------
['massachusetts general hospital: cutaneous biology center']  :  ['nyu', 'new york university']
-------
['edmond safra center for ethics', 'faculty aide']  :  ['fas', 'harvard', 'harvard university']
-------
['nyu rusk institute of rehabilitation medicine', 'instituto de efectividad clínica y sanitaria']  :  ['tufts', 'tufts university']
-------
['albany medical college', 'neural stem cell institute']  :  ['binghamton', 'state university of new york at binghamton']
-------
['johns hopkins som']  :  ['jhu'

['university of texas at arlington']  :  ['dartmouth', 'dartmouth college']
-------
['sackler faculty of medicine, tel aviv university']  :  ['nyu', 'new york university']
-------
['christiaen laboratory, center for developmental genetics']  :  ['nyu', 'new york university']
-------
['nan']  :  ['nyu', 'new york university']
-------
['new york university langone medical center']  :  ['bc', 'boston college']
-------
['westchester medical center']  :  ['nyu', 'new york university']
-------
['california pacific medical center', 'university of california, san francisco']  :  ['brown', 'brown university']
-------
["brigam and women's hospital"]  :  ['bio', 'umass', 'university of massachusetts at amherst']
-------
["brigam and women's hospital"]  :  ['cns', 'umass', 'university of massachusetts at amherst']
-------
['joslin diabetes center']  :  ['bc', 'boston college']
-------
['denver health medical center']  :  ['ucdenver']
-------
['sea semester', 'natick soldier research development an

["boston children's hospital", 'umass boston biophysics department', 'harvard medical school department of bioinformatics']  :  ['uconn', 'university of connecticut']
-------
["yale children's hospital"]  :  ['bc', 'boston college']
-------
['nan']  :  ['upenn', 'university of pennsylvania']
-------
['novartis institutes for biomedical research (nibr)']  :  []
-------
['marsh-armstrong lab at kennedy krieger institute']  :  ['jhu', 'johns hopkins university']
-------
['dartmouth psychiatric research center', "cincinnati children's hospital"]  :  ['bennington', 'bennington college']
-------
['johns hopkins center for imaging science', 'rutgers cancer institute of new jersey', 'johns hopkins department of biomedical engineering']  :  ['jhu', 'johns hopkins university']
-------
['md anderson cancer center', 'lwigale lab']  :  ['rice', 'rice university']
-------
['national institutes of health', 'ileana cristea virology and proteomics lab']  :  ['princeton', 'princeton university']
-------

In [15]:
one = ['university of massachusetts boston', 'washington university school of medicine']
two = ['wustl', 'washington university, saint louis', 'washington , saint louis']

# for o in one:
#     for t in two:
#         if t in o:
#         print(t, o)
        
any(substring in two for substring in one)

False